In [ ]:
import matplotlib.pyplot as plt
import ipyvuetify as v
import ipywidgets as ipw
from IPython.display import FileLink
from IPython.display import Javascript

import numpy as np
from PIL import Image
import random
import io

from tqdm import tqdm
from IPython.display import clear_output

In [ ]:
file_uploader = ipw.FileUpload(
    accept='.png,.jpg,.jpeg,.gif',
    multiple=False,
    description="Click to Upload"
)

display_anom_btn = v.Btn(
    color="success",
    class_='ma-2',
    children=[
        v.Icon(left=True, children=[
            'mdi-chart-timeline-variant'
        ]),
        'Display Data Cleaning'
    ],
    disabled=True
)

scale_factor_field = v.TextField(
    type='Number',
    outlined=True,
    v_model=2,
    label='Scale Factor'
)

contrast_field = v.TextField(
    type='Number',
    outlined=True,
    v_model=0,
    label='Contrast Shift'
)

#button to display forecast
process_img_btn = v.Btn(
    color="success",
    class_='ma-2',
    children=[
        v.Icon(left=True, children=[
            'mdi-chart-timeline-variant'
        ]),
        'Process Image'
    ],
    disabled=True
)

clear_btn = v.Btn(
    color="error",
    class_='ma-2',
    children=[
        v.Icon(left=True, children=[
            'mdi-delete-forever'
        ]),
        'Clear Output'
    ]
)

figure_display_area = ipw.Output(
    layout={
        'align-items':'center'
    },
)

upload_caption_area = ipw.Output(
    layout={
        'align-items':'left'
    }
)

# All the plots and output go here
display_col = v.Col(
    tag='div',
    cols=9,
    children=[
        figure_display_area
    ]
)

# All the controls are in one column
upload_caption = ""
controls_col = v.Col(
    tag='div',
    cols=3,
    children=[
        scale_factor_field,
        contrast_field,
        process_img_btn,
        clear_btn
    ]
)

# Everything together in a row
full_display = v.Row(
    tag='div',
    fluid=True,
    children=[
        controls_col,
        display_col
    ]
)

In [ ]:
# Business Logic/Image Processing

def quantilize(value, scale_factor):
    """ 
    Returns the proper number of output pixels to fill in
    for a given darkness of input.
    """
    return int(round((value // (256/scale_factor**2)) + 1))

def color_pixels(output_arr, original_x, original_y, lightness, scale_factor=2, additive_contrast=0):
    """
    Given an original pixel and a lightness, fills in the appropriate amount of
    pixels (these are the "stipples") in the output array.
    """
    
    # Change lightness to up contrast
    if lightness < 256/2:
        l = max(0, lightness - additive_contrast)
    else:
        l = min(255, lightness + additive_contrast)
    
    f = np.zeros(scale_factor**2)
    indices = random.sample(range(0, len(f)), quantilize(l, scale_factor))
    f[indices] = 1
    
    new_sub = np.stack(np.split(f, scale_factor))
    
    output_arr[original_x:original_x+scale_factor, original_y:original_y+scale_factor] = new_sub
    return len(indices)

In [ ]:

# Graphics/UI and direct handler functions

def render():
    """
    Becuase the content of some UI elements depends on the settings in others,
    the UI needs to be re-displayed whenever something is changed.
    
    This gets called once at program start, and then again after every setting change.
    """
    
    #display(util_picker) # No util picker for demo.
    display(file_uploader)
    display(upload_caption_area)
    display(full_display)
    
def process_img_on_click(widget, event, data):
    global input_img
    global input_arr
    
    scale_factor = scale_factor_field.v_model
    additive_contrast = int(contrast_field.v_model)
    
    stipple_count = 0
    
    output_shape = (input_arr.shape[0]*scale_factor, input_arr.shape[1]*scale_factor)
    output_arr = np.zeros(output_shape)
    
    with figure_display_area:
        
        # Plot input image and GS version
        fig, (ax1, ax2) = plt.subplots(1, 2)
        ax1.set_xticks([])
        ax1.set_yticks([])
        ax1.imshow(input_img)
        ax1.set_title("Input Image")
        
        ax2.set_xticks([])
        ax2.set_yticks([])
        ax2.imshow(input_arr, cmap="gray")
        ax2.set_title("Greyscaled")
        plt.show()
        
        for i in tqdm(range(0, input_arr.shape[0]), desc="Processing Image"):
            for j in range(0, input_arr.shape[1]):
                stipple_count += color_pixels(output_arr, i, j, input_arr[i][j], scale_factor, additive_contrast)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(output_arr[0:input_arr.shape[0],0:input_arr.shape[1]],cmap="gray")
        plt.imsave("out.png", output_arr[0:input_arr.shape[0],0:input_arr.shape[1]],cmap="gray")
        plt.show()

        print("Stipples Used:", stipple_count)

# Have to wrap the clear fn to take care of the param
def clear_btn_on_click(widget, event, data):
    plt.close('all')
    figure_display_area.clear_output()

In [ ]:
# Data Reader
def file_uploader_on_change(change):
    global input_arr
    global input_img
    
    [filename] = file_uploader.value
    uploaded_file = file_uploader.value[filename]
    size = uploaded_file["metadata"]["size"]
    file_uploader.description = filename
    
    input_bytes = io.BytesIO(uploaded_file["content"])
    input_img = Image.open(input_bytes).convert("RGBA")
    input_grey = input_img.convert('L')
    input_arr = np.asarray(input_grey)
    
    process_img_btn.disabled = False
    
    # Print status
    upload_caption_area.clear_output()
    with upload_caption_area:
        print(f"Loaded: {filename}\nSize: {input_arr.shape[0]}x{input_arr.shape[1]}")
    

In [ ]:
# Configure on_click and on_change behaviors
process_img_btn.on_event('click', process_img_on_click)
clear_btn.on_event('click', clear_btn_on_click)
file_uploader.observe(file_uploader_on_change, 'value')

# Dummy globals
input_arr = None
input_img = None

render()